# 1. Import packages

In [1]:
!pip3 install loralib

In [2]:
import time
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2.Data Loader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

BATCH_SIZE = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


100%|██████████| 170498071/170498071 [00:16<00:00, 10605594.06it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 3. LoRA

In [4]:
class LoRA_Layer():
    def __init__(
        self,
        r: int,
        lora_alpha: int,
        lora_dropout: float,
        merge_weights: bool,
      ):
        self.r = r
        self.lora_alpha = lora_alpha
        if lora_dropout > 0:
            self.lora_dropout = nn.Dropout(p=lora_dropout)
        else:
            self.lora_dropout = lambda x: x
        self.merged = False
        self.merge_weights = merge_weights



In [5]:
class LoRA_Linear(nn.Linear, LoRA_Layer):
    def __init__(self, in_features: int, out_features: int, r: int = 0,
                 lora_alpha: int = 1, lora_dropout: float =0., merge_weights: bool =True, **kwargs):
        nn.Linear.__init__(self, in_features, out_features, **kwargs)
        LoRA_Layer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout, merge_weights=merge_weights)

        if r > 0:
            self.lora_A = nn.Parameter(self.weight.new_zeros((r, in_features)))
            self.lora_B = nn.Parameter(self.weight.new_zeros((out_features, r)))
            self.scaling = self.lora_alpha / self.r
            self.weight.requires_grad = False  # disable gradient of the original weights, from nn.Linear
        self.reset_parameters()

        def reset_parameters(self):
            nn.Linear.reset_parameters(self) # init kaiming hee uniform weights
            if hasattr(self, 'lora_A'):
                nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
                nn.init.zeros_(self.lora_B)

        def train(self, mode:bool = True):
            nn.Linear.train(self, mode)

            # Training mode: subtract lora from the weights if they are merged
            if mode:
                if self.merge_weights and self.merged:
                    if self.r > 0:
                        self.weight.data -= (self.lora_B @ self.lora_A) * self.scaling
                    self.merged = False

            # Evaluation mode: adds lora to the weights if they are not merged
            else:
                if self.merge_weights and not self.merged:
                    if self.r > 0:
                        self.weight.data += (self.lora_B @ self.lora_A) * self.scaling
                    self.merged = True

        def forward(self, x: torch.Tensor):
            # Evaluation mode
            if self.r > 0 and not self.merged:
                result = F.linear(x, self.weight, bias=self.bias)
                result += (self.lora_dropout(x) @ self.lora_A.transpose(0, 1) @ self.lora_B.transpose(0, 1)) * self.scaling
                return result

            # Training mode
            else:
                return F.linear(x, self.weight, bias=self.bias)


# 3. Modeling



In [6]:
def trainable_params(model):
    total_params = 0
    for param in model.parameters():
        if param.requires_grad:
            total_params += param.numel()

    print(f"Trainable params: {total_params/1e6:.2f} M")

## 3.1 Load pre_trained VGG16 Imagene1K



In [7]:
from torchvision.models import vgg16, VGG16_Weights
vgg16_model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

vgg16_classifier_ckpt = vgg16_model.classifier.state_dict()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 84.1MB/s]


In [8]:
trainable_params(vgg16_model)

Trainable params: 138.36 M


In [9]:
trainable_params(vgg16_model.classifier)

Trainable params: 123.64 M


## 3.2 Define clf with LORA, feature extractor the same

In [10]:
lora_classifier =  nn.Sequential(
    LoRA_Linear(in_features=25088, out_features=4096, bias=True, r=16),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    LoRA_Linear(in_features=4096, out_features=4096, bias=True, r=16),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    LoRA_Linear(in_features=4096, out_features=1000, bias=True, r=16)
)

## 3.3: Load pretrained weights to LORA_CLF

In [11]:
lora_classifier.load_state_dict(vgg16_classifier_ckpt, strict=False)

_IncompatibleKeys(missing_keys=['0.lora_A', '0.lora_B', '3.lora_A', '3.lora_B', '6.lora_A', '6.lora_B'], unexpected_keys=[])

## 3.4: Add a layer with output features is 10 for CIFAR10

In [12]:
new_classififer = nn.Sequential(
    *lora_classifier,
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    LoRA_Linear(in_features=1000, out_features=10, bias=True)
)

In [13]:
trainable_params(new_classififer)

Trainable params: 0.70 M


## 3.5 Wrap pre-trained VGG16 features (will be frozen) with LoRA classifier

In [14]:
class CLS_model(nn.Module):
    def __init__(self)->None:
        super().__init__()
        self.features = vgg16_model.features.eval()
        for param in self.features.parameters():
            param.requires_grad_(False)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = new_classififer

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [15]:
model = CLS_model().to(device)

In [16]:
trainable_params(model)

Trainable params: 0.70 M


# 4. Training baby

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [18]:
model.train()

start = time.time()
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{10}], Average Loss: {running_loss / len(trainloader):.4f}, GPU used: {torch.cuda.memory_allocated(0)/1e9:.2f} G')

print('Finished Training')
print(f'Training time: {time.time() - start:.2f} s')

Epoch [1/10], Average Loss: 1.7969, GPU used: 0.58 G
Epoch [2/10], Average Loss: 1.4651, GPU used: 0.58 G
Epoch [3/10], Average Loss: 1.4387, GPU used: 0.58 G
Epoch [4/10], Average Loss: 1.4310, GPU used: 0.58 G
Epoch [5/10], Average Loss: 1.4288, GPU used: 0.58 G
Epoch [6/10], Average Loss: 1.4216, GPU used: 0.58 G
Epoch [7/10], Average Loss: 1.4173, GPU used: 0.58 G
Epoch [8/10], Average Loss: 1.4115, GPU used: 0.58 G
Epoch [9/10], Average Loss: 1.4077, GPU used: 0.58 G
Epoch [10/10], Average Loss: 1.3978, GPU used: 0.58 G
Finished Training
Training time: 209.59 s


# 5. Eval


In [19]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 56.61 %
